## Repliikide eraldamine ja andmebaasi loomine test50

In [1]:
from estnltk import Text
import re
import regex
from estnltk.taggers.system.rule_taggers import RegexTagger, Ruleset
from estnltk.taggers.system.rule_taggers import StaticExtractionRule
from estnltk.taggers import HeaderBasedSegmenter
from estnltk.taggers import NerTagger
import sqlite3
import os
import datetime
from estnltk.converters import text_to_json

Esiteks võetakse üks näidisfail:

In [3]:
# Näitefailis puudub kõneosal punktuatsioon
test_file = open("syntax_analysis_data/2013-03-09.txt", mode="r", encoding="utf-8")
test_text = test_file.read()
print(test_text[:300])

K1: ühe kindla kuju.
K2: Asi ja kongressi sõna härra aga aga Siim nii ei tee.
Ja asi vask ESA Taal Lonni talus öös.
See ei ole see ei saa olla nii et bussi ees.
K3: Ja ta oli majandus.
K4: Tervist sinu Kuku raadio Keskpäevatund Tallinna stuudios Ainar Ruussaar ja Priit olemegi Tartus Hannes Astok ja


In [4]:
print(re.findall(r'^.',test_text))
print(re.findall(r'((^|\n).)',test_text))
print(re.findall(r'((^|\n).*:)',test_text))

['K']
[('K', ''), ('\nK', '\n'), ('\nJ', '\n'), ('\nS', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nP', '\n'), ('\nS', '\n'), ('\nK', '\n'), ('\nü', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nT', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nT', '\n'), ('\nS', '\n'), ('\nL', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nE', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nJ', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nJ', '\n'), ('\nP', '\n'), ('\nN', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nP', '\n'), ('\nK', '\n'), ('\nP', '\n'), ('\nO', '\n'), ('\nM', '\n'), ('\nK', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nK', '\n'), ('\nN', '\n'), ('\nK', '\n'), ('\nS', '\n'), ('\nA', '\n'), ('\nS', '\n'), ('\nE', '\n'), ('\nK', '\n'), ('\nM', '\n'), ('\nK',

Kõnelejale vastav regulaaravaldis:

In [6]:
pattern = r'((^|\n)(?P<span>(?P<name>.*): ?))'

In [7]:
print([match.group('name') for match in re.finditer(pattern, test_text)])
print([match.group('span') for match in re.finditer(pattern, test_text)])
print([match.group() for match in re.finditer(pattern, test_text)])

['K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K4', 'K7', 'K4', 'K6', 'K5', 'K4', 'K6', 'K4', 'K5', 'K7', 'K6', 'K7', 'K5', 'K4', 'K5', 'K4', 'K7', 'K4', 'K5', 'K4', 'K6', 'K4', 'K7', 'K4', 'K7', 'K4', 'K5', 'K7', 'K4', 'K6', 'K5', 'K6', 'K7', 'K4', 'K6', 'K7', 'K4', 'K6', 'K4', 'K7', 'K5', 'K6', 'K5', 'K6', 'K7', 'K5', 'K4', 'K5', 'K4', 'K7', 'K6', 'K5', 'K7', 'K5', 'K6', 'K5', 'K4', 'K5', 'K6', 'K5', 'K4', 'K6', 'K4', 'K8', 'K4', 'K5', 'K4', 'K6', 'K4', 'K5', 'K7', 'K5', 'K6', 'K7', 'K4', 'K5', 'K6', 'K7', 'K4', 'K5', 'K6', 'K4', 'K6', 'K4', 'K7', 'K4', 'K5', 'K6', 'K7', 'K6', 'K4', 'K5', 'K6', 'K4', 'K7', 'K5', 'K7', 'K4', 'K7', 'K4', 'K5', 'K6', 'K7', 'K5', 'K7', 'K4', 'K6', 'K4', 'K6', 'K4']
['K1: ', 'K2: ', 'K3: ', 'K4: ', 'K5: ', 'K6: ', 'K4: ', 'K7: ', 'K4: ', 'K6: ', 'K5: ', 'K4: ', 'K6: ', 'K4: ', 'K5: ', 'K7: ', 'K6: ', 'K7: ', 'K5: ', 'K4: ', 'K5: ', 'K4: ', 'K7: ', 'K4: ', 'K5: ', 'K4: ', 'K6: ', 'K4: ', 'K7: ', 'K4: ', 'K7: ', 'K4: ', 'K5: ', 'K7: ', 'K4: ', 'K6: ', 'K5: ', 'K6: '

Dekoraator kõneleja nime atribuudi lisamiseks:

In [8]:
def decorator(layer,span,annotation):
    annotation['person'] = annotation['match'].group(4)
    return annotation

Lisatakse reegel kõneleja fikseeriva tekstifragmendi väljavõtmiseks:

In [9]:
rule = [StaticExtractionRule(pattern=regex.Regex('((^|\n)(?P<span>(?P<name>.*): ?))'), group=3)]
ruleset = Ruleset()
ruleset.add_rules(rule)

Märgendaja RegexTagger loomine:

In [10]:
tagger = RegexTagger(
    ruleset = ruleset,
    output_layer = 'headers',
    output_attributes = ['person'],
    decorator=decorator,
    match_attribute='match'
)

Katsetus näidisteksti peal:

In [11]:
tagged_test_text = tagger.tag(Text(test_text))
display(tagged_test_text.headers[:5])

Layer(name='headers', attributes=('person',), spans=SL[Span('K1: ', [{'person': 'K1'}]),
Span('K2: ', [{'person': 'K2'}]),
Span('K3: ', [{'person': 'K3'}]),
Span('K4: ', [{'person': 'K4'}]),
Span('K5: ', [{'person': 'K5'}])])

Märgendaja HeaderBasedSegmenter loomine:

In [12]:
tagger2 = HeaderBasedSegmenter(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person'][0]},
    include_header = False
)

In [13]:
tagger2.tag(tagged_test_text)
tagged_test_text.lines[:5]

Layer(name='lines', attributes=('person',), spans=SL[Span('ühe kindla kuju.\n', [{'person': 'K1'}]),
Span('Asi ja kongressi sõna härra aga aga Siim nii ei tee.\nJa asi vask ESA Taal Lonni talus öös.\nSee ei ole see ei saa olla nii et bussi ees.\n', [{'person': 'K2'}]),
Span('Ja ta oli majandus.\n', [{'person': 'K3'}]),
Span('Tervist sinu Kuku raadio Keskpäevatund Tallinna stuudios Ainar Ruussaar ja Priit olemegi Tartus Hannes Astok ja miks.\nPärast laulis Singer Vinger et raske tabaksid siiski silmile kindlasti on olemas üks inimene kes neid näeb see on meie esiuus juht Taavi Laur ekspankur ja tema.\nSuured rasked rahapakid noh tema palgaks mäe on olnud pangas nägema tema aasta I siis nõukogu heakskiidu või nõukogu ettepanekul siis oma uueks palganumbriks kuus tuhat viis eurot kuus mis siis vanas rahas on umbes sada tuhat.\nKrooni kuus noh aasta välisvanasti ja.\nüks koma kaks miljonit aastas üks koma kaks miljonit ütleme niimoodi et sa pead maksud maha umbes kaheksakümmend tuhat ja sisse kuus kätte ja noh ütleme nii et kui mõistlikult majandada siis iga kuu peaks umbes nii-öelda kogu tippmajandus umbes viiskümmend tuhat viiskümmend tuhat krooni vanas aastaks lihtsalt.\nEt arve peale üle jääma ehk etteastega kuskil niimoodi pool miljonit.\nKasvatada nii-öelda hoiukassas jääb on kas endale endale sellist on rasvakihti on kindlasti väga mugav.\nAga see teema on äratanud suurt vastukaja sellel teemal on asunud võitlusse omavahel isegi sellised olulised isikud riigis nagu peaminister Andrus Ansipi ja siis Reformierakonna oluline tegelane mõjutaja me ei saada.\n', [{'person': 'K4'}]),
Span('No nüüd rohkem Tõnis Palts Valdo ka all ja ja veel mitmed ettevõtlusega seotud inimesed on mul sel teemal kus on avatud ja kõik nad on kõik aga millegipärast rünnanud peamiste Ansipit öelnud et.\nAnsipi väite et jah võib-olla peaks riigiettevõtete juhtide ja sihtasutuse juhtide palkasid kuidagi kuidagi piirama või kastist oma lauspopulism.\n', [{'person': 'K5'}])])

Nimeüksuste märgendaja NerTagger:

In [14]:
nertagger = NerTagger()

Luuakse andmebaas ja seal uus tabel, kuhu lisatakse kõik repliikidele vastavad Text-objektid, samuti info kuupäeva, kõneleja nime ja repliigi numbri kohta.

In [18]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE lines(ID INTEGER PRIMARY KEY, line TEXT, number INTEGER, person TEXT, date DATE)")

In [19]:
path = 'syntax_analysis_data/'
for file in os.listdir(path):
    with open(os.path.join(path, file), mode="r", encoding="utf-8") as f:
        raw_text = f.read()
        filename = file.replace('.txt', '')
        date = datetime.datetime.strptime(filename, "%Y-%m-%d").date()
        
        text = tagger.tag(Text(raw_text))
        text = tagger2.tag(text)
        
        for i in range(len(text.lines)):
            span = text.lines[i]
            line = span.text.strip()
            if line == '':
                continue
            number = i+1
            line = Text(line)
            line.meta['line_number'] = number
            line.meta['person'] = span.person
            line.meta['date'] = filename
            line.tag_layer(['sentences', 'morph_analysis'])
            nertagger.tag(line)
            line_json = text_to_json(line)
            cur.execute("""INSERT INTO lines
                                  (line, number, person, date) 
                                  VALUES (?, ?, ?, ?);""", (line_json, number, span.person, date))
            con.commit()

In [20]:
con.close()